In [1]:
from globals_and_helpers import (
    PROJECT_DIR,
    TEMP_OUTPUTS_DIR,
    FINAL_OUTPUTS_DIR,
 )
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

datapath = os.path.join(PROJECT_DIR, 'final_outputs', '0.25 scaling v. tiled')

# Load the CSV files into dataframes
df1_unfiltered = pd.read_csv(os.path.join(datapath, '27620_0.25scale.csv'))
df2_unfiltered = pd.read_csv(os.path.join(datapath, '27620_tiled.csv'))
df2_unfiltered.head()

,Object ID,Object type,Classification,Mean Intensity,Unnamed: 4,Mean Intensity Normalized,Stiffness (log (E actual) PA)
0,f4dd3722-9b0b-45be-82b1-b5701390fb23,Annotation,LGD,117.4291,NaN,0.460506,5.752603
1,25ed0478-a9f2-43cf-9c2f-ce5d7d535ab5,Annotation,LGD,100.5337,NaN,0.394250,5.411505
2,57151830-f97e-40c8-816c-6fb8f179df44,Annotation,LGD,86.3118,NaN,0.338478,5.124381
3,a4e29dc8-2683-40ed-8cdc-bc6d3a21c992,Annotation,LGD,119.8190,NaN,0.469878,5.800853
4,778a383f-2284-4ea9-9d2d-f7e9a4b88e4d,Annotation,LGD,112.1996,NaN,0.439998,5.647026


In [2]:
df2 = df2_unfiltered[['Object ID', 'Object type', 'Classification', 'Stiffness (log (E actual) PA)']]
df1 = df1_unfiltered[['Object ID', 'Object type', 'Classification', 'Stiffness (log (E actual) PA)']]
df1.head()

,Object ID,Object type,Classification,Stiffness (log (E actual) PA)
0,7a0bd252-1529-4c0c-818a-4c171b75ea22,Annotation,LGD,6.848749
1,29e685e5-3482-447f-9160-93966e2c41e6,Annotation,LGD,6.985607
2,344fd8bc-fd5a-4da9-87eb-593c62c8c603,Annotation,LGD,6.416180
3,86ff07ca-9b93-4f62-98c2-34c39b8801c4,Annotation,LGD,6.895183
4,96eb9cec-1b6f-42cb-8487-58ec2e186404,Annotation,LGD,6.701627


In [3]:
df2.head()

,Object ID,Object type,Classification,Stiffness (log (E actual) PA)
0,f4dd3722-9b0b-45be-82b1-b5701390fb23,Annotation,LGD,5.752603
1,25ed0478-a9f2-43cf-9c2f-ce5d7d535ab5,Annotation,LGD,5.411505
2,57151830-f97e-40c8-816c-6fb8f179df44,Annotation,LGD,5.124381
3,a4e29dc8-2683-40ed-8cdc-bc6d3a21c992,Annotation,LGD,5.800853
4,778a383f-2284-4ea9-9d2d-f7e9a4b88e4d,Annotation,LGD,5.647026


In [4]:
# pd.set_option('display.max_rows', None)  # Show all rows
df_combine = pd.merge(df1, df2, on=['Object ID', 'Object type', 'Classification'], suffixes=(' (1/4 scaled)', ' (tiled)'))
df_combine.head()

,Object ID,Object type,Classification,Stiffness (log (E actual) PA) (1/4 scaled),Stiffness (log (E actual) PA) (tiled)
0,7a0bd252-1529-4c0c-818a-4c171b75ea22,Annotation,LGD,6.848749,5.368817
1,29e685e5-3482-447f-9160-93966e2c41e6,Annotation,LGD,6.985607,5.966017
2,344fd8bc-fd5a-4da9-87eb-593c62c8c603,Annotation,LGD,6.416180,5.126315
3,86ff07ca-9b93-4f62-98c2-34c39b8801c4,Annotation,LGD,6.895183,5.384763
4,96eb9cec-1b6f-42cb-8487-58ec2e186404,Annotation,LGD,6.701627,5.199659


In [5]:
# Perform an outer join to include all rows from both DataFrames
merged_df = pd.merge(df1, df2, on=['Object ID', 'Object type', 'Classification'], how='outer', suffixes=(' (1/4 scaled)', ' (tiled)'), indicator=True)

# Filter rows that did not merge
unmerged_rows = merged_df[merged_df['_merge'] != 'both']

# Display the unmerged rows
unmerged_rows.to_csv(os.path.join(datapath, '27620_unmerged_rows.csv'), index=False)

In [6]:
print(df1.describe())
print(df2.describe())

       Stiffness (log (E actual) PA)
count                      60.000000
mean                        6.526701
std                         0.314872
min                         6.013916
25%                         6.206861
50%                         6.579188
75%                         6.797549
max                         7.119532
       Stiffness (log (E actual) PA)
count                      81.000000
mean                        5.573291
std                         0.492706
min                         4.476879
25%                         5.229292
50%                         5.584434
75%                         5.855532
max                         7.121520


In [7]:
from plotly.offline import iplot
import plotly.graph_objects as go
trace = go.Scatter(x=df1['Stiffness (log (E actual) PA)'], y=df2['Stiffness (log (E actual) PA)'], mode="markers")
data = [trace]
layout = go.Layout(title="Correlation of Stiffness Values", xaxis=dict(title="1/4 Scaled"), yaxis=dict(title="Tiled"))
iplot({"data": data, "layout": layout})

In [8]:
import plotly.express as px
import plotly.io as pio
import statsmodels.api as sm
pio.templates.default = 'none'

# px.scatter(df_combine, x='Stiffness (log (E actual) PA) (1/4 scaled)', y='Stiffness (log (E actual) PA) (tiled)')

# Create the scatter plot with 'Object ID' in hover data
fig = px.scatter(
    df_combine,
    x='Stiffness (log (E actual) PA) (1/4 scaled)',
    y='Stiffness (log (E actual) PA) (tiled)',
    hover_data=['Object ID'],  # Include 'Object ID' in hover data
    trendline="ols"  # Add a regression line
)

# Show the plot
fig.show()

In [11]:
trace0 = go.Histogram(
    x=df1['Stiffness (log (E actual) PA)'], 
    name='1/4 Scaled', 
    opacity=0.75,
    nbinsx=100
)
trace1 = go.Histogram(
    x=df2['Stiffness (log (E actual) PA)'], 
    name='Tiled', 
    opacity=0.75,
    nbinsx=100
)
data = [trace0, trace1]
layout = go.Layout(title="Distribution of Stiffness Values", xaxis=dict(title="Stiffness (log (E actual) PA)"), yaxis=dict(title="Frequency"))
iplot({"data": data, "layout": layout})

In [15]:
import plotly.graph_objs as go
from plotly.offline import iplot
import numpy as np
from scipy.stats import gaussian_kde

# Create histogram traces with a specified number of bins
trace0 = go.Histogram(
    x=df1['Stiffness (log (E actual) PA)'],
    name='1/4 Scaled',
    opacity=0.75,
    nbinsx=100  # Specify the number of bins
)
trace1 = go.Histogram(
    x=df2['Stiffness (log (E actual) PA)'],
    name='Tiled',
    opacity=0.75,
    nbinsx=100  # Specify the number of bins
)

# Create KDE traces
kde0 = go.Scatter(
    x=np.linspace(min(df1['Stiffness (log (E actual) PA)']), max(df1['Stiffness (log (E actual) PA)']), 200),
    y=gaussian_kde(df1['Stiffness (log (E actual) PA)'])(np.linspace(min(df1['Stiffness (log (E actual) PA)']), max(df1['Stiffness (log (E actual) PA)']), 200)),
    mode='lines',
    name='1/4 Scaled KDE',
    line=dict(color='blue', width=2)
)
kde1 = go.Scatter(
    x=np.linspace(min(df2['Stiffness (log (E actual) PA)']), max(df2['Stiffness (log (E actual) PA)']), 200),
    y=gaussian_kde(df2['Stiffness (log (E actual) PA)'])(np.linspace(min(df2['Stiffness (log (E actual) PA)']), max(df2['Stiffness (log (E actual) PA)']), 200)),
    mode='lines',
    name='Tiled KDE',
    line=dict(color='orange', width=2)
)

# Combine the traces into a data list
data = [trace0, trace1, kde0, kde1]

# Define the layout
layout = go.Layout(
    title="Distribution of Stiffness Values with KDE",
    xaxis=dict(title="Stiffness (log (E actual) PA)"),
    yaxis=dict(title="Frequency")
)

# Create the figure and plot
fig = go.Figure(data=data, layout=layout)
iplot(fig)


In [ ]:
from scipy.stats import spearmanr

# Perform Spearman's Rank Correlation
correlation, p_value = spearmanr(
    df_combine['Stiffness (log (E actual) PA) (1/4 scaled)'], 
    df_combine['Stiffness (log (E actual) PA) (tiled)']
)

print(f"Spearman's Rank Correlation: {correlation}")
print(f"P-value: {p_value}")


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 60 and the array at index 1 has size 81